In [ ]:
%%writefile create_veterans_parallel.py

import math
import ipyparallel
import itertools

# at terminal : ipcluster start -n 4

clients = ipyparallel.Client()
dview = clients.direct_view()

with dview.sync_imports():
    from mouseworld import mouseworld
    from mouseworld.mouseworld import Mouseworld
    import time
dview.push({"Mouseworld": Mouseworld})

def run_experiment() :
#    
    num_mice = [0, 0, 100]

    # Build the model
    print('Building mouseworld')
    model = mouseworld.Mouseworld(num_mice, 100, 0, mouse_initial_energy = 1000, mouse_max_energy = 1200,
                     food_amount_range = (20,40), nutritional_value = [1], mouse_reproduction = False, brain_iterations_per_step = 10)

    # Gather initial randomly distributed data
    # model.initial_datacollector.collect(model,model.schedule)
    # initial_model_data = model.initial_datacollector.get_model_vars_dataframe()
    # initial_model_data.to_csv('results/initial_model_data.csv', sep='\t')

    # Prepare environment by stepping food and predators and diffusing odors
    print('Preparing environment')
    for i in range(40) :
        model.food_schedule.step()
        model.predator_schedule.step()
        model.diffuse_odor_layers(model.odor_layers)

    counter = 0
    myrange = 60
    # Run for discrete number of timesteps
#     print('Simulatimg for %i timesteps'%myrange)
#     for i in range(myrange) :
#         c=time.time()
#         counter += 1
#         model.step()
#         d=time.time()
#         print('sim step : %i in %f'%(counter, d-c))
#     print('Simulation terminated - Number of time steps reached')

    # Run until all mice perish
    print('Simulatimg until all mice perish')
    while model.num_mice > 0 :
        c=time.time()
        counter += 1
        model.step()
        d=time.time()
        print('sim step : %i in %f'%(counter, d-c))
    print('Simulation terminated - No alive mice')
        
    # Gather final model and agent data
    model.final_datacollector.collect(model,model.all_mice_schedule)
    # final_model_data = model.final_datacollector.get_model_vars_dataframe()
    # print(final_model_data)

    final_agent_data = model.final_datacollector.get_agent_vars_dataframe()
    mouse_statistics = final_agent_data[['first_action_duration', 'first_action_termination']]
    # mouse_statistics = final_agent_data['action_history']
    # mouse_statistics = final_agent_data['secondary_values']
    # mouse_statistics = final_agent_data['sensor_vector']

    mouse_statistics = mouse_statistics.reset_index('Step', drop = True)
        #mouse_statistics = mouse_statistics.reset_index('AgentID', drop = True)
    num_trials = len(mouse_statistics.index)
    # for i in range(num_trials)
    succesful_trials = mouse_statistics.loc[(mouse_statistics['first_action_termination'] == 'Closure')]
    unsuccesful_trials = mouse_statistics.loc[(mouse_statistics['first_action_termination'] == 'Failure')]
    # incomplete_trials = mouse_statistics.loc[(mouse_statistics['first_action_termination'] == 'None')]

    num_succesful_trials = len(succesful_trials.index)
    num_unsuccesful_trials = len(unsuccesful_trials.index)
    # num_incomplete_trials = len(incomplete_trials.index)

    #performance = num_succesful_trials / num_trials
    mean_time_success = succesful_trials['first_action_duration'].mean()
    mean_time_failure = unsuccesful_trials['first_action_duration'].mean()

    sim_params = [mouse_position, num_mice, myrange]
    results = [num_succesful_trials, mean_time_success, num_unsuccesful_trials, mean_time_failure]
    exp_data = [results, sim_params]
    return exp_data
    # mouse_statistics = [performance,  mean_time]
    #     sensor_vector = final_agent_data['sensor_vector'][0].values[0]
#     sensor_position = final_agent_data['sensor_position'][0].values[0]
#     motor_vector = final_agent_data['motor_vector'][0].values[0]
#     first_action = final_agent_data['action_history'][0].values[0].loc[0]
#     first_action = mousetest_data
#     first_action = (mousetest_data['Duration'], mousetest_data['Termination'])
    #first_action = final_agent_data['action_history'][0].values[0].loc[0]
#     return (performance,  mean_time)
#     return (mouse_statistics)
# print(performance,  mean_time)
all_exp_data = dview.map_sync(run_experiment, mouse_positions)

file = open('results/check_approach_consistency_001_2.txt','w') 
for exp_data in all_exp_data :
    file.write(str(exp_data) + '\n')
# file.write(str(results) + '\n')
file.close() 
# mouse_statistics.to_csv('results/mouse_statistics.csv', sep='\t')

In [3]:
%%writefile create_veterans.py

from mouseworld import mouseworld
import time

simulation_num = sys.argv[1]

num_mice = [0, 0, 10]

 # Build the model
print('Building mouseworld')
model = mouseworld.Mouseworld(num_mice, 100, 0, simulation_num, mouse_initial_energy = 10000, mouse_max_energy = 12000,
                 food_amount_range = (20,40), nutritional_value = [1], mouse_reproduction = False, 
                              brain_iterations_per_step = 10, mousebrain_seed = 8)

# Gather initial randomly distributed data
# model.initial_datacollector.collect(model,model.schedule)
# initial_model_data = model.initial_datacollector.get_model_vars_dataframe()
# initial_model_data.to_csv('results/initial_model_data.csv', sep='\t')

# Prepare environment by stepping food and predators and diffusing odors
print('Preparing environment')
for i in range(40) :
    model.food_schedule.step()
    model.predator_schedule.step()
    model.diffuse_odor_layers(model.odor_layers)

counter = 0
myrange = 2
# Run for discrete number of timesteps
# print('Simulatimg for %i timesteps'%myrange)
# for i in range(myrange) :
#     c=time.time()
#     counter += 1
#     model.step()
#     d=time.time()
#     print('sim step : %i in %f'%(counter, d-c))
# print('Simulation terminated - Number of time steps reached')


# Run until all mice perish
print('Simulatimg until all mice perish')
while model.num_mice > 0 :
    c=time.time()
    counter += 1
    model.step()
    d=time.time()
    print('sim step : %i in %f'%(counter, d-c))
print('Simulation terminated - No alive mice')

print('Storing mousebrains')
# print(model.exp_approach_rank)
# print(model.exp_approach_rank[:1])
for i in model.exp_approach_rank[:5] :
    i.store_mousebrain_weights()
    
# # Gather final model and agent data
# model.final_datacollector.collect(model,model.all_mice_schedule)
# # final_model_data = model.final_datacollector.get_model_vars_dataframe()
# # print(final_model_data)

# final_agent_data = model.final_datacollector.get_agent_vars_dataframe()
# mouse_statistics = final_agent_data[['first_action_duration', 'first_action_termination']]


Overwriting create_veterans.py


In [2]:
%%writefile test_veterans_parallel.py

import math
import ipyparallel
import itertools
import os
import numpy as np
# at terminal : ipcluster start -n 4

clients = ipyparallel.Client()
dview = clients.direct_view()

print('Loading modules to engines') 
with dview.sync_imports():
    from mouseworld import mouseworld
    from mouseworld.mouseworld import Mouseworld
    import time
    import sys
    import numpy
dview.push({"Mouseworld": Mouseworld})
print('Defining filenames')

directory = ('results/simulation_%s/veteran_mousebrains'%sys.argv[1])

def run_experiment(file) :

    mouse_data = numpy.load(file)

    if mouse_data['motor_NN_on'] :
        if mouse_data['learning_on'] :
            num_mice = [0, 0, 152]
        else :
            num_mice = [0, 152, 0]
    else :
        num_mice = [152, 0, 0]
    genome = numpy.around(mouse_data['genome'], decimals = 2)
    genome_range = [(x,x) for x in genome]
    mousebrain_seed = mouse_data['seed']
    brain_iterations_per_step = mouse_data['brain_iterations_per_step']
    mousebrain_steps = mouse_data['mousebrain_steps']
    mousebrain_weights = [mouse_data['w_search'], mouse_data['w_approach'], mouse_data['w_avoid']]
    simulation_num = mouse_data['simulation_num']

    # Build the model
    print('Building mouseworld')
    model = mouseworld.Mouseworld(num_mice, 1, 0, simulation_num, genome_range = genome_range, 
                                  mouse_position = 'in_quadrant', food_position = (0,0), 
                                  primary_values = [100, 0], secondary_values = [100,0], food_amount_range = (10000,10000), 
                                  nutritional_value = [1], mouse_reproduction = False, 
                                  brain_iterations_per_step = brain_iterations_per_step, mousebrain_seed = mousebrain_seed,
                                 initial_mousebrain_weights = mousebrain_weights, test_veteran = True)

    # Gather initial randomly distributed data
    # model.initial_datacollector.collect(model,model.schedule)
    # initial_model_data = model.initial_datacollector.get_model_vars_dataframe()
    # initial_model_data.to_csv('results/initial_model_data.csv', sep='\t')

    # Prepare environment by stepping food and predators and diffusing odors
    print('Preparing environment')
    for i in range(40) :
        model.food_schedule.step()
        model.predator_schedule.step()
        model.diffuse_odor_layers(model.odor_layers)

    counter = 0
    myrange = 40
    # Run for discrete number of timesteps
    print('Simulatimg for %i timesteps'%myrange)
    for i in range(myrange) :
        c=time.time()
        counter += 1
        model.step()
        d=time.time()
        print('sim step : %i in %f'%(counter, d-c))
    print('Simulation terminated - Number of time steps reached')


    # Run until all mice perish
    # print('Simulatimg until all mice perish')
    # while model.num_mice > 0 :
    #     c=time.time()
    #     counter += 1
    #     model.step()
    #     d=time.time()
    #     print('sim step : %i in %f'%(counter, d-c))
    # print('Simulation terminated - No alive mice')

    # Gather final model and agent data
    print('Gathering simulation data')
    model.final_datacollector.collect(model,model.all_mice_schedule)

    final_agent_data = model.final_datacollector.get_agent_vars_dataframe()
    mouse_statistics = final_agent_data[['control_population', 'first_action_duration', 'first_action_termination']]

    mouse_statistics = mouse_statistics.reset_index('Step', drop = True)
    print('Counting successful trials')
    veteran_stats = mouse_statistics.loc[mouse_statistics['control_population'] == False]
    control_stats = mouse_statistics.loc[mouse_statistics['control_population'] == True]
    num_trials = len(veteran_stats.index)
    succesful_veteran_trials = veteran_stats.loc[(veteran_stats['first_action_termination'] == 'Closure')]
    succesful_control_trials = control_stats.loc[(control_stats['first_action_termination'] == 'Closure')]
    unsuccesful_veteran_trials = veteran_stats.loc[(veteran_stats['first_action_termination'] == 'Failure')]
    unsuccesful_control_trials = control_stats.loc[(control_stats['first_action_termination'] == 'Failure')]

    num_succesful_veteran_trials = len(succesful_veteran_trials.index)
    num_succesful_control_trials = len(succesful_control_trials.index)                                     
    num_unsuccesful_veteran_trials = len(unsuccesful_veteran_trials.index)
    num_unsuccesful_control_trials = len(unsuccesful_control_trials.index)

    mean_time_veteran_success = succesful_veteran_trials['first_action_duration'].mean()
    mean_time_control_success = succesful_control_trials['first_action_duration'].mean()                 
    mean_time_veteran_failure = unsuccesful_veteran_trials['first_action_duration'].mean()
    mean_time_control_failure = unsuccesful_control_trials['first_action_duration'].mean()

    veteran_results = [num_succesful_veteran_trials, mean_time_veteran_success, num_unsuccesful_veteran_trials, mean_time_veteran_failure]
    control_results = [num_succesful_control_trials, mean_time_control_success, num_unsuccesful_control_trials, mean_time_control_failure]

    exp_data = [mousebrain_steps, mousebrain_seed, veteran_results, control_results]
    return exp_data

filenames = []
for file in os.listdir(directory):
    if file.endswith(".npz"):
        filenames.append('%s/%s'%(directory,file))
        
print('Initializing parallel execution')      
all_exp_data = dview.map_sync(run_experiment, filenames)

print('Writing results to file')
file = open('%s/test_veterans.txt'%directory,'w') 
counter = 0
for exp_data in all_exp_data :
    counter +=1
    mousebrain_steps = exp_data[0]
    mousebrain_seed = exp_data[1]
    veteran_results = exp_data[2]
    control_results = exp_data[3]
    file.write('-----Veteran Mouse : %i-----\n'%counter)
    file.write('experience : %s'%str(mousebrain_steps) + '\t')
    file.write('mousebrain seed : %s' %str(mousebrain_seed) + '\n')
    file.write('veteran results' + '\t')
    file.write(str(veteran_results) + '\n')
    file.write('control results' + '\t')
    file.write(str(control_results) + '\n')

file.close() 


Overwriting test_veterans_parallel.py


In [11]:
%%writefile test_veterans.py

from mouseworld import mouseworld
import time
import sys
import numpy as np

mouse_data = np.load(sys.argv[1])

if mouse_data['motor_NN_on'] :
    if mouse_data['learning_on'] :
        num_mice = [0, 0, 152]
    else :
        num_mice = [0, 152, 0]
else :
    num_mice = [152, 0, 0]
genome = np.around(mouse_data['genome'], decimals = 2)
genome_range = [(x,x) for x in genome]
mousebrain_seed = mouse_data['seed']
brain_iterations_per_step = mouse_data['brain_iterations_per_step']
mousebrain_steps = mouse_data['mousebrain_steps']
mousebrain_weights = [mouse_data['w_search'], mouse_data['w_approach'], mouse_data['w_avoid']]
                 
# Build the model
print('Building mouseworld')
model = mouseworld.Mouseworld(num_mice, 1, 0, genome_range = genome_range, 
                              mouse_position = 'in_quadrant', food_position = (0,0), 
                              primary_values = [100, 0], secondary_values = [100,0], food_amount_range = (10000,10000), 
                              nutritional_value = [1], mouse_reproduction = False, 
                              brain_iterations_per_step = brain_iterations_per_step, mousebrain_seed = mousebrain_seed,
                             initial_mousebrain_weights = mousebrain_weights, test_veteran = True)

# Gather initial randomly distributed data
# model.initial_datacollector.collect(model,model.schedule)
# initial_model_data = model.initial_datacollector.get_model_vars_dataframe()
# initial_model_data.to_csv('results/initial_model_data.csv', sep='\t')

# Prepare environment by stepping food and predators and diffusing odors
print('Preparing environment')
for i in range(40) :
    model.food_schedule.step()
    model.predator_schedule.step()
    model.diffuse_odor_layers(model.odor_layers)

counter = 0
myrange = 40
# Run for discrete number of timesteps
print('Simulatimg for %i timesteps'%myrange)
for i in range(myrange) :
    c=time.time()
    counter += 1
    model.step()
    d=time.time()
    print('sim step : %i in %f'%(counter, d-c))
print('Simulation terminated - Number of time steps reached')


# Run until all mice perish
# print('Simulatimg until all mice perish')
# while model.num_mice > 0 :
#     c=time.time()
#     counter += 1
#     model.step()
#     d=time.time()
#     print('sim step : %i in %f'%(counter, d-c))
# print('Simulation terminated - No alive mice')
    
# Gather final model and agent data
print('Gathering simulation data')
model.final_datacollector.collect(model,model.all_mice_schedule)

final_agent_data = model.final_datacollector.get_agent_vars_dataframe()
mouse_statistics = final_agent_data[['control_population', 'first_action_duration', 'first_action_termination']]

mouse_statistics = mouse_statistics.reset_index('Step', drop = True)
print('Counting successful trials')
veteran_stats = mouse_statistics.loc[mouse_statistics['control_population'] == False]
control_stats = mouse_statistics.loc[mouse_statistics['control_population'] == True]
num_trials = len(veteran_stats.index)
succesful_veteran_trials = veteran_stats.loc[(veteran_stats['first_action_termination'] == 'Closure')]
succesful_control_trials = control_stats.loc[(control_stats['first_action_termination'] == 'Closure')]
unsuccesful_veteran_trials = veteran_stats.loc[(veteran_stats['first_action_termination'] == 'Failure')]
unsuccesful_control_trials = control_stats.loc[(control_stats['first_action_termination'] == 'Failure')]

num_succesful_veteran_trials = len(succesful_veteran_trials.index)
num_succesful_control_trials = len(succesful_control_trials.index)                                     
num_unsuccesful_veteran_trials = len(unsuccesful_veteran_trials.index)
num_unsuccesful_control_trials = len(unsuccesful_control_trials.index)

mean_time_veteran_success = succesful_veteran_trials['first_action_duration'].mean()
mean_time_control_success = succesful_control_trials['first_action_duration'].mean()                 
mean_time_veteran_failure = unsuccesful_veteran_trials['first_action_duration'].mean()
mean_time_control_failure = unsuccesful_control_trials['first_action_duration'].mean()

veteran_results = [num_succesful_veteran_trials, mean_time_veteran_success, num_unsuccesful_veteran_trials, mean_time_veteran_failure]
control_results = [num_succesful_control_trials, mean_time_control_success, num_unsuccesful_control_trials, mean_time_control_failure]

file = open('results/veteran_mousebrains/test_veterans.txt','w') 
file.write('experience : %s'%str(mousebrain_steps) + '\t')
file.write('mousebrain seed : %s' %str(mousebrain_seed) + '\n')
file.write('veteran results' + '\t')
file.write(str(veteran_results) + '\n')
file.write('control results' + '\t')
file.write(str(control_results) + '\n')

file.close() 


Overwriting test_veterans.py
